In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
import random
import shutil

In [2]:
def get_size(image_path):
    image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
    image = np.array(image)
    image = image.astype('float32') / 255
    return image.shape

In [3]:
def num_images(image_path):
    count = 0
    for dir1 in os.listdir(image_path):
        count += len(os.listdir(os.path.join(image_path, dir1)))
    return count

In [4]:
image_height = get_size('train_images/video_0/0.jpg')[0]    # 720
image_width = get_size('train_images/video_0/0.jpg')[1]     # 1280
print(f"Height: {image_height}, Width: {image_width}")
print(f"Training data size: {num_images('train_images')}")  # 23501

Height: 720, Width: 1280
Training data size: 23501


In [5]:
def create_dataset(image_folder):
    image_data = []
    class_names = []

    for dir1 in os.listdir(image_folder):
        for dir2 in os.listdir(os.path.join(image_folder, dir1)):
            image_path = os.path.join(image_folder, dir1, dir2)
            image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (image_height, image_width), interpolation = cv2.INTER_AREA)
            image = np.array(image)
            image = image.astype('float32') / 255
            image_data.append(image)
            class_names.append(dir1)

    return image_data

In [6]:
def get_random_images(image_folder):
    image_paths = []
    while (len(image_paths) < 10):
        for dir1 in os.listdir(image_folder):
            for dir2 in os.listdir(os.path.join(image_folder, dir1)):
                image_path = os.path.join(image_folder, dir1, dir2)
                if (random.randint(0,10000) > 9999 and len(image_paths) < 10):
                    image_paths.append(image_path)

    return image_paths

In [7]:
def store_images(image_paths):
    os.makedirs('test_images')
    for i in range(0, len(image_paths)):
        shutil.copy(image_paths[i], 'test_images')

In [14]:
%cd ..

/Users/joeljose/Desktop/Coding Projects/python/comp9517/project/Crown-of-Throrns


In [8]:
test_images = get_random_images('train_images')
store_images(test_images)
print(test_images)

['train_images/video_2/9860.jpg', 'train_images/video_1/5743.jpg', 'train_images/video_1/4032.jpg', 'train_images/video_2/5869.jpg', 'train_images/video_0/9950.jpg', 'train_images/video_0/11619.jpg', 'train_images/video_2/9520.jpg', 'train_images/video_2/562.jpg', 'train_images/video_2/10677.jpg', 'train_images/video_0/11708.jpg']


In [9]:
import wandb

try:
    api_key = "f72dca236cb3497e771bf6c5ea119c25fcfb1b10"
    wandb.login(key=api_key)
    anonymous = None
    print('Logged in!')
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: justjo3l. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/joeljose/.netrc


Logged in!


In [10]:
run = wandb.init()
artifact = run.use_artifact('justjo3l/YOLOR/run_1w24g32k_model:v0', type='model')
artifact_dir = artifact.download()
print(artifact_dir)

wandb: Downloading large artifact run_1w24g32k_model:v0, 315.54MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.2


./artifacts/run_1w24g32k_model:v0


In [11]:
!ls

README.md            test.csv             wandb
artifacts            test_images          yolov7.ipynb
detect.py            train.csv
hog_svm_method.ipynb train_images


In [12]:
!rm -r yolov7
!git clone https://github.com/WongKinYiu/yolov7.git # clone
%cd yolov7
%pip install -qr requirements.txt  # install

rm: yolov7: No such file or directory
Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (467/467), done.
/Users/joeljose/Desktop/Coding Projects/python/comp9517/project/Crown-of-Throrns/yolov7
Note: you may need to restart the kernel to use updated packages.


In [13]:
%cd yolov7

[Errno 2] No such file or directory: 'yolov7'
/Users/joeljose/Desktop/Coding Projects/python/comp9517/project/Crown-of-Throrns/yolov7


Replace the detect.py in yolov7 with the detect.py in the Crown-Of-Throrns repository

In [15]:
!python detect.py --weights ../artifacts/run_1w24g32k_model/best.pt --conf 0.1 --source ../test_images

Namespace(weights=['../artifacts/run_1w24g32k_model/best.pt'], source='../test_images', img_size=640, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 2022-11-15 torch 1.13.0 CPU

Fusing layers... 
IAuxDetect.fuse
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
/Users/joeljose/opt/anaconda3/envs/cotsenv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 819 layers, 164816216 parameters, 0 gradients, 225.4 GFLOPS
 Convert model to Traced-model... 
 traced_script_module